# JSON Query Engine
The JSON query engine is useful for querying JSON documents that conform to a JSON schema.

This JSON schema is then used in the context of a prompt to convert a natural language query into a structured JSON Path query. This JSON Path query is then used to retrieve data to answer the given question.

In [1]:
# First, install the jsonpath-ng package which is used by default to parse & execute the JSONPath queries.
!pip install jsonpath-ng

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import dotenv
dotenv.load_dotenv("../../../.env")

False

In [4]:
from IPython.display import Markdown, display

### Let's start on a Toy JSON

Very simple JSON object containing data from a blog post site with user comments.

We will also provide a JSON schema (which we were able to generate by giving ChatGPT a sample of the JSON).

#### Advice
Do make sure that you've provided a helpful `"description"` value for each of the fields in your JSON schema.

As you can see in the given example, the description for the `"username"` field mentions that usernames are lowercased. You'll see that this ends up being helpful for the LLM in producing the correct JSON path query.

In [5]:
# Test on some sample data 
json_value = {
  "blogPosts": [
    {
      "id": 1,
      "title": "First blog post",
      "content": "This is my first blog post"
    },
    {
      "id": 2,
      "title": "Second blog post",
      "content": "This is my second blog post"
    }
  ],
  "comments": [
    {
      "id": 1,
      "content": "Nice post!",
      "username": "jerry",
      "blogPostId": 1
    },
    {
      "id": 2,
      "content": "Interesting thoughts",
      "username": "simon",
      "blogPostId": 2
    },
    {
      "id": 3,
      "content": "Loved reading this!",
      "username": "simon",
      "blogPostId": 2
    }
  ]
}

# JSON Schema object that the above JSON value conforms to
json_schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "description": "Schema for a very simple blog post app",
  "type": "object",
  "properties": {
    "blogPosts": {
      "description": "List of blog posts",
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": {
            "description": "Unique identifier for the blog post",
            "type": "integer"
          },
          "title": {
            "description": "Title of the blog post",
            "type": "string"
          },
          "content": {
            "description": "Content of the blog post",
            "type": "string"
          }
        },
        "required": ["id", "title", "content"]
      }
    },
    "comments": {
      "description": "List of comments on blog posts",
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": {
            "description": "Unique identifier for the comment",
            "type": "integer"
          },
          "content": {
            "description": "Content of the comment",
            "type": "string"
          },
          "username": {
            "description": "Username of the commenter (lowercased)",
            "type": "string"
          },
          "blogPostId": {
            "description": "Identifier for the blog post to which the comment belongs",
            "type": "integer"
          }
        },
        "required": ["id", "content", "username", "blogPostId"]
      }
    }
  },
  "required": ["blogPosts", "comments"]
}


In [6]:
from llama_index.indices.service_context import ServiceContext
from langchain.llms.openai import OpenAI
from llama_index.indices.struct_store import JSONQueryEngine

llm = OpenAI(model_name="text-davinci-003")
service_context = ServiceContext.from_defaults()
nl_query_engine = JSONQueryEngine(json_value=json_value, json_schema=json_schema, service_context=service_context)
raw_query_engine = JSONQueryEngine(json_value=json_value, json_schema=json_schema, service_context=service_context, synthesize_response=False)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
NumExpr defaulting to 2 threads.


/workspaces/llama_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
nl_response = nl_query_engine.query(
    "What comments has Jerry been writing?",
)
raw_response = raw_query_engine.query(
    "What comments has Jerry been writing?",
)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 797 tokens
> [query] Total LLM token usage: 797 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 363 tokens
> [query] Total LLM token usage: 363 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens


In [8]:
display(Markdown(f"<h1>Natural language Response</h1><br><b>{nl_response}</b>"))
display(Markdown(f"<h1>Raw JSON Response</h1><br><b>{raw_response}</b>"))

<h1>Natural language Response</h1><br><b> Jerry has written one comment with the content 'Nice post!' on blog post with id 1.</b>

<h1>Raw JSON Response</h1><br><b>[{"id": 1, "content": "Nice post!", "username": "jerry", "blogPostId": 1}]</b>

In [9]:
# get the json path query string. Same would apply to raw_response
print(nl_response.extra_info["json_path_response_str"])

 $.comments[?(@.username == 'jerry')]
